# All Statistics 8.5.3

Create a sample of size 100 from normal distribution, and use DKW inequality to create a 95% confidence band from the sample. Then check wither the normal cdf lies in the confidence band.

Do this 1000 times to see what the confidence actually is of the constructed confidence bands.

Doing this experiment once gave a confidence of 0.948.
Standard error of this confidence approximately equals sqrt(0.948*(1-0.948)/1000) = 0.007.

In [ ]:
from tqdm.notebook import tqdm

import math
import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt

In [ ]:
np.random.seed(0)

In [ ]:
class StepwiseFunction():
    """
    Objects of this class are functions defined as follows:
    given x0<...<xn and y0,...,y(n+1):
    -if xi <= x < x(i+1), then f(x) = y(i+1)
    -if x < x0, then f(x) = y0
    -if xn <= x, then f(x) = y(n+1)
    
    Created to be right continuous as the class is designed for
    cumulative distributive functions of discrete distributions
    
    Parameters
    ----------
    input_cutoffs
        ordered list of values at which the
        functions' values change / list of discontinuities of
        the function
    output_values
        list of output values of the function
    """
    def __init__(self, input_cutoffs, output_values):
        self.input_cutoffs = input_cutoffs
        self.output_values = output_values
    
    def __call__(self, x):
        """
        writing self_cutoffs as x0<...<xn and
        output_values as y0,...,y(n+1), define function so that:
        -if xi <= x < x(i+1), then f(x) = y(i+1)
        -if x < x0, then f(x) = y0
        -if xn <= x, then f(x) = y(n+1)

        Created to be right continuous as the class is designed for
        cumulative distributive functions of discrete distributions
        """
        for index in range(len(self.input_cutoffs)):
            if x < self.input_cutoffs[index]:
                return self.output_values[index]
            else:
                continue
        
        # if function has not yet returned, then x >= xn
        return self.output_values[-1]
    
    def plot(self, ax, padding = 2, color = 'blue'):
        """
        plot the function represented by self onto the provided axis
        
        Parameters
        ----------
        ax
            the axis object in which the plots should be added
        padding
            how far below x0 and above xn we should plot
        color
            color of new data to be added to the plot
        
        Returns
        -------
        ax
            axis with new data added to it
        """
        xs = self.input_cutoffs
        ys = self.output_values
        
        # plot regions between x0 and xn
        for i in range(0, len(xs)-1):
            x = [xs[i], xs[i+1]]
            y = [ys[i+1], ys[i+1]]
            ax.plot(x, y, color=color)
            
        # plot region below x0 and region above xn
        ax.plot([xs[0] - padding, xs[0]], [ys[0], ys[0]], color = color)
        ax.plot([xs[-1], xs[-1] + [padding]], [ys[-1], ys[-1]], color = color)
        
        # plot points to indicate the value of the function at the cutoffs
        ax.scatter(xs, ys[1:], color=color, s=10)
        
        return ax
    
    def is_smaller_than_cdf(self, f):
        """
        determine if object is smaller than f for all inputs
        
        Parameters
        ----------
        f
            a strictly increasing continuous function bounded between 0 and 1
            i.e. a cdf of a continuous distribution
        """
        xs = self.input_cutoffs
        ys = self.output_values
        
        # note that ys[i+1] = self(xs[i])
        for i in range(len(xs)):
            if ys[i+1] >= f(xs[i]):
                return False
            else:
                continue
        
        # the for loop above deals with x >= x0
        # need separate if loop for case x < x0
        if ys[0] <= 0:
            return True
        else:
            return False
    
    def is_bigger_than_cdf(self, f):
        """
        determine if object is smaller than f for all inputs
        
        Parameters
        ----------
        f
            a strictly increasing continuous function bounded between 0 and 1
            i.e. a cdf of a continuous distribution
        """
        xs = self.input_cutoffs
        ys = self.output_values
        
        # note that ys[i] = self(xs[i-1])
        for i in range(len(xs)):
            if ys[i] <= f(xs[i]):
                return False
            else:
                continue
        
        # the for loop above deals with x <= xn
        # need separate if loop for case x > xn
        if ys[-1] >= 1:
            return True
        else:
            return False
        

### testing stepwise function class

In [ ]:
xs = np.arange(5)
ys = np.arange(6)

test = StepwiseFunction(xs, ys)

fig, ax = plt.subplots()
ax = test.plot(ax, color = 'red', padding = 5)

In [ ]:
def create_empirical_dist_fn(sample):
    """
    takes as input a sample, and outputs the empirical distribution
    function of the sample
    """
    sample.sort()
    one_over_n = 1/len(sample)
    input_cutoffs = [sample[0]]
    output_values = [0, one_over_n]
    
    for x in sample[1:]:
        if x != input_cutoffs[-1]:
            input_cutoffs.append(x)
            output_values.append(output_values[-1] + one_over_n)
        else:
            output_values[-1] += one_over_n
    
    return StepwiseFunction(input_cutoffs = input_cutoffs, output_values = output_values)

### test create empirical distribution function

In [ ]:
sample = np.random.normal(size = 20)
print(sample)

Fn = create_empirical_dist_fn(sample)
fig, ax = plt.subplots()
Fn.plot(ax)

In [ ]:
def create_confidence_band(sample, alpha=0.05):
    """
    given a sample, produces a confidence band for the cdf of the generating
    distribution. width of the band is determined using the dkw inequality
    
    Parameters
    ----------
    sample
        list or numpy array. sample from some distribution
    alpha
        the false positive rate. Or, output is 1-alpha confidence band
    
    Returns
    -------
    Ln
        StepwiseFunction. The lower bound of the 1-alpha confidence
        band for the cdf
    
    Un
        StepwiseFunction. The upper bound of the 1-alpha confidence
        band for the cdf
    
    """
    n = len(sample)
    Fn = create_empirical_dist_fn(sample)
    eps_n = (math.log(2/alpha)/2/n)**0.5
    
    # create Ln
    Ln_input_cutoffs = Fn.input_cutoffs
    Ln_output_values = []
    
    for y in Fn.output_values:
        Ln_output_values.append(max(0, y-eps_n))
    
    Ln = StepwiseFunction(input_cutoffs = Ln_input_cutoffs,
                          output_values = Ln_output_values                        
                         )
    
    # create Un
    Un_input_cutoffs = Fn.input_cutoffs
    Un_output_values = []
    
    for y in Fn.output_values:
        Un_output_values.append(min(1, y+eps_n))
    
    Un = StepwiseFunction(input_cutoffs = Un_input_cutoffs,
                          output_values = Un_output_values                        
                         )
    
    return Ln, Un

### test confidence band creation

In [ ]:
sample = np.random.normal(size = 20)
print(sample)

Fn = create_empirical_dist_fn(sample)
Ln, Un = create_confidence_band(sample)
fig, ax = plt.subplots()
Fn.plot(ax, color='black')
Ln.plot(ax, color='red')
Un.plot(ax, color='blue')

### test stepwise function less than

In [ ]:
sample = np.random.normal(size = 20)

Ln, Un = create_confidence_band(sample, alpha = 0.8)

print(Ln.is_smaller_than_cdf(norm.cdf))

# plot to visually check it is sensible
fig, ax = plt.subplots()
Ln.plot(ax, color='red')

x = np.arange(Ln.input_cutoffs[0]-2, Ln.input_cutoffs[-1] + 2, 0.1)
norm_x = norm.cdf(x)
ax.plot(x, norm_x)

### test stepwise function bigger than

In [ ]:
sample = np.random.normal(size = 20)

Ln, Un = create_confidence_band(sample, alpha = 0.8)

print(Un.is_bigger_than_cdf(norm.cdf))

# plot to visually check it is sensible
fig, ax = plt.subplots()
Un.plot(ax, color='red')

x = np.arange(Ln.input_cutoffs[0]-2, Ln.input_cutoffs[-1] + 2, 0.1)
norm_x = norm.cdf(x)
ax.plot(x, norm_x)

In [ ]:
def create_and_test_bands(sample_size, num_samples, alpha = 0.05):
    num_bands_containing_cdf = 0
    
    for _ in range(num_samples):
        sample = np.random.normal(size = sample_size)
        Ln, Un = create_confidence_band(sample, alpha = alpha)
        
        if Ln.is_smaller_than_cdf(norm.cdf) and Un.is_bigger_than_cdf(norm.cdf):
            num_bands_containing_cdf += 1
    
    return num_bands_containing_cdf

In [ ]:
create_and_test_bands(100, 1000)